In [ ]:
import cv2
import numpy as np

# Global variable to store calibrated color values
calibrated_colors = {}

def get_dominant_color(roi):
    """
    Get the dominant color in a region of interest
    Uses the center portion to avoid edge artifacts
    """
    h, w = roi.shape[:2]
    # Take center 60% of the region to avoid edges
    center_roi = roi[int(h*0.2):int(h*0.8), int(w*0.2):int(w*0.8)]
    
    # Convert to HSV
    hsv_roi = cv2.cvtColor(center_roi, cv2.COLOR_BGR2HSV)
    
    # Get average HSV values
    avg_color = cv2.mean(hsv_roi)[:3]
    
    return avg_color

def classify_color(hsv_color):
    """
    Classify HSV color based on calibrated values
    Returns color name and BGR value for display
    """
    if not calibrated_colors:
        return 'unknown', (128, 128, 128)
    
    h, s, v = hsv_color
    
    # Check white first (low saturation)
    if s < 50 and v > 150:
        return 'white', (255, 255, 255)
    
    # Find closest calibrated color
    min_distance = float('inf')
    closest_color = 'unknown'
    
    for color_name, (cal_h, cal_s, cal_v) in calibrated_colors.items():
        # Calculate distance in HSV space
        # Hue is circular, so we need special handling
        h_diff = min(abs(h - cal_h), 180 - abs(h - cal_h))
        s_diff = abs(s - cal_s)
        v_diff = abs(v - cal_v)
        
        # Weighted distance (hue is most important)
        distance = (h_diff * 2) + (s_diff * 0.5) + (v_diff * 0.3)
        
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name
    
    # Color BGR values for display
    color_bgr_map = {
        'white': (255, 255, 255),
        'yellow': (0, 255, 255),
        'orange': (0, 165, 255),
        'red': (0, 0, 255),
        'green': (0, 255, 0),
        'blue': (255, 0, 0)
    }
    
    return closest_color, color_bgr_map.get(closest_color, (128, 128, 128))

def detect_face_colors(frame, grid_rect):
    """
    Detect colors in a 3x3 grid within the specified rectangle
    """
    x, y, w, h = grid_rect
    
    # Calculate size of each cell
    cell_w = w // 3
    cell_h = h // 3
    
    colors = []
    color_names = []
    
    for row in range(3):
        row_colors = []
        row_names = []
        for col in range(3):
            # Calculate cell boundaries
            cell_x = x + col * cell_w
            cell_y = y + row * cell_h
            
            # Extract ROI for this cell
            roi = frame[cell_y:cell_y+cell_h, cell_x:cell_x+cell_w]
            
            # Get dominant color
            hsv_color = get_dominant_color(roi)
            
            # Classify the color
            color_name, color_bgr = classify_color(hsv_color)
            
            row_colors.append(color_bgr)
            row_names.append(color_name)
        
        colors.append(row_colors)
        color_names.append(row_names)
    
    return color_names, colors

def draw_grid(frame, grid_rect, detected_colors=None, color_bgrs=None):
    """
    Draw a 3x3 grid on the frame with detected colors
    """
    x, y, w, h = grid_rect
    
    cell_w = w // 3
    cell_h = h // 3
    
    # Draw grid lines
    for i in range(4):
        # Vertical lines
        cv2.line(frame, (x + i * cell_w, y), (x + i * cell_w, y + h), (0, 255, 0), 2)
        # Horizontal lines
        cv2.line(frame, (x, y + i * cell_h), (x + w, y + i * cell_h), (0, 255, 0), 2)
    
    # Draw detected colors if available
    if detected_colors and color_bgrs:
        for row in range(3):
            for col in range(3):
                cell_x = x + col * cell_w
                cell_y = y + row * cell_h
                
                # Draw color indicator
                color_bgr = color_bgrs[row][col]
                color_name = detected_colors[row][col]
                
                # Draw filled circle with detected color
                center_x = cell_x + cell_w // 2
                center_y = cell_y + cell_h // 2
                cv2.circle(frame, (center_x, center_y), 15, color_bgr, -1)
                cv2.circle(frame, (center_x, center_y), 15, (0, 0, 0), 2)
                
                # Draw color name
                text_size = cv2.getTextSize(color_name[0].upper(), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
                text_x = center_x - text_size[0] // 2
                text_y = cell_y + cell_h - 10
                cv2.putText(frame, color_name[0].upper(), (text_x, text_y),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

def calibrate_colors():
    """
    Calibrate all 6 colors by showing them to the camera
    """
    global calibrated_colors
    calibrated_colors = {}
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera for calibration")
        return False
    
    # Set camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    frame_w, frame_h = 1280, 720
    
    # Smaller calibration square in center
    cal_size = 150
    cal_rect = (
        (frame_w - cal_size) // 2,
        (frame_h - cal_size) // 2,
        cal_size,
        cal_size
    )
    
    colors_to_calibrate = ['white', 'yellow', 'orange', 'red', 'green', 'blue']
    
    print("\n" + "="*60)
    print("COLOR CALIBRATION")
    print("="*60)
    print("Show each color sticker in the center square one at a time.")
    print("Make sure the sticker fills the square and is well-lit.")
    print("Press SPACE when the color is positioned correctly.")
    print("="*60 + "\n")
    
    for color_name in colors_to_calibrate:
        print(f"\nCalibrating {color_name.upper()}...")
        print(f"Show a {color_name.upper()} sticker in the center square and press SPACE")
        
        captured = False
        samples = []
        
        while not captured:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame")
                cap.release()
                cv2.destroyAllWindows()
                return False
            
            display_frame = frame.copy()
            
            # Draw calibration square
            x, y, w, h = cal_rect
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 255), 3)
            
            # Get current color in the square
            roi = frame[y:y+h, x:x+w]
            hsv_color = get_dominant_color(roi)
            
            # Display instructions
            cv2.putText(display_frame, f"Calibrating: {color_name.upper()}", (10, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
            cv2.putText(display_frame, "Position sticker in yellow square", (10, 90),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(display_frame, "Press SPACE to capture", (10, 130),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(display_frame, f"HSV: H={hsv_color[0]:.0f} S={hsv_color[1]:.0f} V={hsv_color[2]:.0f}",
                       (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
            
            # Show progress
            progress = f"Color {colors_to_calibrate.index(color_name) + 1}/{len(colors_to_calibrate)}"
            cv2.putText(display_frame, progress, (frame_w - 150, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            
            cv2.imshow('Color Calibration', display_frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == 32:  # Space bar
                # Take multiple samples for stability
                for i in range(10):
                    ret, frame = cap.read()
                    if ret:
                        roi = frame[y:y+h, x:x+w]
                        samples.append(get_dominant_color(roi))
                
                # Average the samples
                avg_hsv = np.mean(samples, axis=0)
                calibrated_colors[color_name] = tuple(avg_hsv)
                
                print(f"  ✓ {color_name.upper()} calibrated: HSV = {avg_hsv}")
                captured = True
            elif key == ord('q'):
                print("Calibration cancelled")
                cap.release()
                cv2.destroyAllWindows()
                return False
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*60)
    print("CALIBRATION COMPLETE!")
    print("="*60)
    print("Calibrated colors:")
    for color_name, hsv in calibrated_colors.items():
        print(f"  {color_name.upper()}: H={hsv[0]:.0f}, S={hsv[1]:.0f}, V={hsv[2]:.0f}")
    print("="*60 + "\n")
    
    return True

def capture_single_face(face_name):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print(f"Error: Could not open camera for {face_name} face")
        return None
    
    # Set camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    print(f"\nShow the {face_name.upper()} face to camera")
    print("Position the cube so all 9 stickers are visible in the green grid")
    print("Press SPACE to capture, Q to quit")
    
    captured_colors = None
    
    # Define grid rectangle (center of frame)
    frame_w, frame_h = 1280, 720
    grid_size = 300
    grid_rect = (
        (frame_w - grid_size) // 2,
        (frame_h - grid_size) // 2,
        grid_size,
        grid_size
    )
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break
        
        # Create display frame
        display_frame = frame.copy()
        
        # Detect colors in real-time
        detected_colors, color_bgrs = detect_face_colors(frame, grid_rect)
        
        # Draw grid with detected colors
        draw_grid(display_frame, grid_rect, detected_colors, color_bgrs)
        
        # Display instructions
        cv2.putText(display_frame, f"Face: {face_name.upper()}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(display_frame, "SPACE: Capture | Q: Quit", (10, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Rubik\'s Cube Capture', display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == 32:  # Space bar
            print(f"\nCapturing {face_name.upper()} face...")
            print("Detected colors:")
            for row in detected_colors:
                print("  ", row)
            captured_colors = detected_colors
            break
        elif key == ord('q'):
            print("Quitting...")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return captured_colors

# Main program
if __name__ == "__main__":
    print("="*60)
    print("RUBIK'S CUBE SOLVER - FACE CAPTURE")
    print("="*60)
    print("\nThis tool will capture all 6 faces of your Rubik's cube.")
    print("First, we'll calibrate the colors, then capture each face.")
    print("\nIMPORTANT: Use consistent, good lighting throughout!")
    print("="*60)
    
    # Step 1: Calibrate colors
    if not calibrate_colors():
        print("Calibration failed. Exiting.")
        exit()
    
    # Step 2: Capture all 6 faces
    print("\nNow let's capture all 6 faces of the cube.")
    print("Standard cube orientation: White top, Green front")
    
    faces = ['white', 'yellow', 'red', 'orange', 'green', 'blue']
    cube_state = {}
    
    for face in faces:
        print(f"\n{'='*60}")
        print(f"Ready to capture {face.upper()} face")
        print(f"{'='*60}")
        
        colors = capture_single_face(face)
        
        if colors is None:
            print(f"Failed to capture {face} face")
            break
        
        cube_state[face] = colors
    
    if len(cube_state) == 6:
        print("\n" + "="*60)
        print("CAPTURE COMPLETE!")
        print("="*60)
        print("\nCube state collected:")
        for face, colors in cube_state.items():
            print(f"\n{face.upper()} face:")
            for row in colors:
                print(f"  {row}")
        
        # Convert to solver format (flat string)
        print("\n" + "="*60)
        print("Cube string format (for solver):")
        print("="*60)
        
        # Standard order: U R F D L B (White, Orange, Green, Yellow, Red, Blue)
        face_order = ['red', 'orange', 'yellow', 'white', 'blue', 'green']
        cube_string = []
        for face in face_order:
            if face in cube_state:
                for row in cube_state[face]:
                    for color in row:
                        cube_string.append(color[0])  # First letter of color
        
        print(" ".join(cube_string))
    else:
        print("\nIncomplete capture. Please try again.")

RUBIK'S CUBE SOLVER - FACE CAPTURE

This tool will capture all 6 faces of your Rubik's cube.
First, we'll calibrate the colors, then capture each face.

IMPORTANT: Use consistent, good lighting throughout!

COLOR CALIBRATION
Show each color sticker in the center square one at a time.
Make sure the sticker fills the square and is well-lit.
Press SPACE when the color is positioned correctly.


Calibrating WHITE...
Show a WHITE sticker in the center square and press SPACE
  ✓ WHITE calibrated: HSV = [ 28.17003704  27.39048148 144.89707407]

Calibrating YELLOW...
Show a YELLOW sticker in the center square and press SPACE
  ✓ YELLOW calibrated: HSV = [ 33.76412346 156.95064198 154.05095062]

Calibrating ORANGE...
Show a ORANGE sticker in the center square and press SPACE
  ✓ ORANGE calibrated: HSV = [  8.03197531 210.51701235 151.66620988]

Calibrating RED...
Show a RED sticker in the center square and press SPACE
  ✓ RED calibrated: HSV = [  3.08997531 210.51087654 127.5184321 ]

Calibrati